In [1]:
from datetime import date


print("Name- Pratyush Mohanty")
print("Roll No-118CS0180")
print("Date- {}".format(date.today()))
print("Machine learning Lab 6")

Name- Pratyush Mohanty
Roll No-118CS0180
Date- 2021-11-04
Machine learning Lab 6


In [2]:
import pandas as pd
import math
import copy
from collections import Counter
from sklearn.model_selection import train_test_split

colNames = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
carData = pd.read_csv(r"C:\Users\Pratyush\Videos\car_evaluation.csv", names = colNames, header = None)
train, test = train_test_split(carData, test_size = 0.2)
df=carData


carData.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
carData.columns

Index(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class'], dtype='object')

In [4]:
carData.describe()

,buying,maint,doors,persons,lug_boot,safety,class
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,vhigh,vhigh,4,4,small,high,unacc
freq,432,432,432,576,576,576,1210


In [5]:
carData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


formula to calculate information entropy:
$$ Ent(S) = \sum_{i=1}^c -p_i * \log_{c}(p_i)$$
gain function to calculate the information gain of an attribute

In [6]:
# func to calculate information entropy of a given Array
def entropy(inputArray, base = 4):
    entropy = 0
    counts = list(Counter(inputArray).values())
    probs = [x/sum(counts) for x in counts]
    for prob in probs:
        if prob != 0:
            entropy += (-prob * math.log(prob, base))
    return entropy

# func to calculate information gain of a given attribute in a dataframe
def gain(data, attribute, initEntropy, classCol = 'class', classes = ['unacc', 'acc', 'good', 'vgood']):
    
    classCounts = {}  #Attributwert: [Anzahl class1, Anzahl class2, ...] i.e. vhigh: [360 , 72, 0, 0]
    instances = data.shape[0]
    gain = initEntropy
    base = len(classes)
    
    for index, row in data.iterrows():
        idx = classes.index(row[classCol])
        attributeValue = row[attribute]
        if attributeValue not in classCounts:
            classCounts[attributeValue] = [1 if i == idx else 0 for i in range(base)]
        else:
            classCounts[attributeValue][idx] += 1       
    
    for key, value in classCounts.items():
        instancesLeft = sum(value)
        probs = [x/instancesLeft for x in value]
        entropy = 0
        for prob in probs:
            if prob != 0:
                entropy -= prob * math.log(prob, base)
        gain -= (instancesLeft/instances) * entropy
    return gain

check for attribute with highest information gain

In [7]:
print ('gain \"buying\"\t =  '+ str(gain(carData, 'buying', entropy(list(carData['class'])))))
print ('gain \"maint\"\t =  '+ str(gain(carData, 'maint', entropy(list(carData['class'])))))
print ('gain \"doors\"\t =  '+ str(gain(carData, 'doors', entropy(list(carData['class'])))))
print ('gain \"persons\"\t =  '+ str(gain(carData, 'persons', entropy(list(carData['class'])))))
print ('gain \"lug_boot\"\t =  '+ str(gain(carData, 'lug_boot', entropy(list(carData['class'])))))
print ('gain \"safety\"\t =  '+ str(gain(carData, 'safety', entropy(list(carData['class'])))))

gain "buying"	 =  0.04822448458480691
gain "maint"	 =  0.0368519734607429
gain "doors"	 =  0.002242858313316054
gain "persons"	 =  0.1098314816699541
gain "lug_boot"	 =  0.015004070623802712
gain "safety"	 =  0.13109217827713193


non binary tree to store decision tree

In [8]:
class Node:
    def __init__(self, name = 'root', data = None, isLeaf = False):
        self.name = name
        self.data = data
        self.childs = {}
        self.isLeaf = isLeaf
    def __str__(self):
        return str(self.name)
    
    def getSibling(self, paths):
        if paths[0] not in self.childs:
            print('getChild: node ' + str(self.data) + ' has no child with path: ' + paths[0])
        elif paths[1:]:
            return self.childs[paths[0]].getSibling(paths[1:])
        else:
            return self.childs[paths[0]]
        
    def addSibling(self, paths, node):
        if paths[0] not in self.childs:
            if paths[1:]:
                print('addSibling: node ' + self.data + ' has no child with path: ' + paths[0])
            else:
                self.childs[paths[0]] = node
                #print('add Sibling ' + str(node) + ' succes')
        else:
            self.childs[paths[0]].addSibling(paths[1:], node)


def printTree(node, level = 0, path = ''):
    if level == 0:
        print('Tree: ' + str(node))
    else:
        print ('\t' * level + '└' + '─' + path + '─' + str(node))
    if node.childs:
        for key in node.childs:
            printTree(node.childs[key], level + 1, key)

# try out if adding childs and printing works
root = Node()

root.addSibling(['a1'], Node('Level 1'))
root.addSibling(['a2'], Node('+'))
root.addSibling(['a1', 'b1'], Node('Level 2'))
root.addSibling(['a1', 'b2'], Node('-'))
root.addSibling(['a1', 'b3'], Node('*'))

printTree(root)

Tree: root
	└─a1─Level 1
		└─b1─Level 2
		└─b2─-
		└─b3─*
	└─a2─+


build up the decision tree

In [9]:
#func to recursivly build up dtree
def buildTree(node, leftAttributes, classCol = 'class', classes = ['unacc', 'acc', 'good', 'vgood']):
    
    # initialize if node is root
    if node.name == 'root':
        #get the attribute with max information gain to build child
        currentEntropy = entropy(list(node.data[classCol]))
        maxGain = 0
        maxGainAttribute = ''
        for attribute in leftAttributes:
            gains = 0
            gains = gain(node.data, attribute, currentEntropy)
            if gains >= maxGain:
                maxGain = gains
                maxGainAttribute = attribute
      
        #build child with highest information gain attribute
        leftAttributes.remove(maxGainAttribute)
        child = Node(maxGainAttribute, node.data)
        node.addSibling([''], child)
        buildTree(node.getSibling(['']), list(leftAttributes))
        
    # else build tree
    else:
        #get unique attribute values
        attribute = node.name
        attributeValues = set(node.data[attribute])
        oldDf = node.data
        
        for attributeValue in attributeValues:    
            #create new data with only required attribute values
            newDf = oldDf.loc[oldDf[attribute] == attributeValue]

            #if new Entropy is 0 its a Leaf
            currentEntropy = entropy(list(newDf[classCol]))
            
            #no leaf: go deeper 
            if currentEntropy != 0 and leftAttributes:           
                maxGain = 0
                maxGainAttribute = ''
                for leftAttribute in leftAttributes:
                    gains = gain(newDf, leftAttribute, currentEntropy)
                    if gains >= maxGain:
                        maxGain = gains
                        maxGainAttribute = leftAttribute
                        
                left = list(leftAttributes)
                left.remove(maxGainAttribute)
                child = Node(maxGainAttribute, newDf)
                node.addSibling([attributeValue], child)
                buildTree(node.getSibling([attributeValue]), list(left))
            
            #leaf: classificate
            else:
                #most common class gets taken
                classification = Counter(newDf[classCol]).most_common(1)[0][0]
                child = Node(classification, isLeaf = True)
                node.addSibling([attributeValue], child)         

                
allAttributes = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']

#initialize root node with training data
root = Node('root', train)
buildTree(root, allAttributes)

printTree(root)

Tree: root
	└──safety
		└─high─persons
			└─4─buying
				└─high─maint
					└─med─acc
					└─vhigh─unacc
					└─low─acc
					└─high─acc
				└─vhigh─maint
					└─high─unacc
					└─vhigh─unacc
					└─low─acc
					└─med─acc
				└─low─maint
					└─high─lug_boot
						└─med─doors
							└─4─vgood
							└─3─acc
							└─5more─vgood
							└─2─acc
						└─small─acc
						└─big─vgood
					└─med─lug_boot
						└─med─doors
							└─4─vgood
							└─2─good
							└─5more─vgood
						└─small─good
						└─big─vgood
					└─low─lug_boot
						└─med─doors
							└─4─vgood
							└─3─good
							└─2─good
							└─5more─vgood
						└─small─good
						└─big─vgood
					└─vhigh─acc
				└─med─maint
					└─med─lug_boot
						└─med─doors
							└─4─vgood
							└─3─acc
							└─2─acc
							└─5more─vgood
						└─small─acc
						└─big─vgood
					└─vhigh─acc
					└─low─lug_boot
						└─med─doors
							└─4─vgood
							└─3─good
							└─2─good
							└─5more─vgood
						└─small─good
						└─big─vgood
					└─hig

start predicting

In [10]:
def predict(node, instance):
    currentAttribute = node.name
    if node.name == 'root':
        return predict(node.getSibling(['']), instance)
    if node.isLeaf:
        return node.name
    else:
        instAttribVal = instance[currentAttribute]
        return predict(node.getSibling([instAttribVal]), instance) 
    
# try some classifications
print (str(test.to_dict('records')[0]) + '\nPREDICTION: class = ' + predict(root, test.to_dict('records')[0]))
print (str(test.to_dict('records')[1]) + '\nPREDICTION: class = ' + predict(root, test.to_dict('records')[1]))

{'buying': 'vhigh', 'maint': 'low', 'doors': '4', 'persons': '2', 'lug_boot': 'small', 'safety': 'low', 'class': 'unacc'}
PREDICTION: class = unacc
{'buying': 'high', 'maint': 'high', 'doors': '4', 'persons': 'more', 'lug_boot': 'med', 'safety': 'low', 'class': 'unacc'}
PREDICTION: class = unacc


In [11]:
y_pred=[]
for i in range(len(test.to_dict('records'))):
    if test.to_dict("records")[i]['class']==predict(root,test.to_dict('records')[0]):
        y_pred.append(0)
    else:
        y_pred.append(1)
        

In [12]:
import numpy as np
np.mean(y_pred)

0.315028901734104

In [13]:
import numpy as np

y=[1 for i in range(len(y_pred))]

1.  sum of squares due to regression (SSR)
2.  sum of squares error (SSE)
3.  sum of squares total (SSTO)
4.  coefficient of determinationR2metric

In [14]:
mean_y = np.mean(y)
mean_y_pred = np.mean(y_pred)

SSR = np.sum((y_pred - mean_y)**2)
SSE = (np.sum(y) - np.sum(y_pred))**2
SST = (np.sum(y) - mean_y)**2
R_2 = 1-(SSE/SST)

print(f"SSR: {round(SSR, 2)}")
print(f"SSE: {round(SSE, 2)}")
print(f"SST: {round(SST, 2)}")
print(f"R_2: {round(R_2, 5)}")

SSR: 237.0
SSE: 56169
SST: 119025.0
R_2: 0.52809


T_Stat

In [15]:
t_val = (abs(mean_y - mean_y_pred)*((df.shape[0])**0.5))/np.sum(y)-np.sum(y_pred)

t_test_val = (1/(2*math.pi)**0.5)*math.exp(-0.5*t_val)
print("t_test_val > 0.05 so we will not reject the hypothesis")
print("t_test value: ", t_test_val/10)

t_test_val > 0.05 so we will not reject the hypothesis
t_test value:  1.7868554490138258e+22


F_Stat

In [16]:
f_val = (np.std(y))**2/(np.std(y_pred))**2

f_test_val = (1/(2*math.pi)**0.5)*math.exp(-0.5*f_val)
print("f_test_val > 0.05, so we will not reject the hypothesis")
print("f_test value: ", f_test_val)

f_test_val > 0.05, so we will not reject the hypothesis
f_test value:  0.3989422804014327
